textClassifierConv реализовал сверточные нейронные сети для классификации предложений - Yoo Kim. Пожалуйста, смотрите этот блог для получения полной информации.(https://richliao.github.io/supervised/classification/2016/11/26/textclassifier-convolutional/) 

In [17]:
from __future__ import print_function
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os

os.environ['KERAS_BACKEND'] = 'theano'

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, Add
from keras.models import Model

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/диплом'

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

data_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/диплом/export.csv', sep=',')
print(data_train.shape)

(2716, 7)


In [3]:
data_train.head()

,N,Тип,Источник,Содержание сообщения,Оригинал сообщения,Дата,Unnamed: 6
0,1,Комментарий к посту/репосту,Группа,"Действительно,кто эту провозгласил примадонной...",0,11.05.2022 11:33:25,NaN
1,2,Комментарий к посту/репосту,Группа,"Схем много,спасибо,но по всем связать пенсии н...",0,10.05.2022 19:13:27,NaN
2,3,Комментарий к посту/репосту,Пользователь,"И найдутся же дураки,кто это чмо купит.",0,10.05.2022 19:01:50,NaN
3,4,Комментарий к посту/репосту,Группа,"Из бисера делаю эти подснежники,а хотелось бы ...",0,10.05.2022 18:53:50,NaN
4,5,Комментарий к посту/репосту,Группа,"Да пристрелить уже этого педика ,из ума выжил ...",0,10.05.2022 18:21:55,NaN


In [4]:
data = pd.DataFrame()
data['content'] = data_train['Содержание сообщения']
data['label'] = data_train['Оригинал сообщения']
data.dropna() 
data.head()

,content,label
0,"Действительно,кто эту провозгласил примадонной...",0
1,"Схем много,спасибо,но по всем связать пенсии н...",0
2,"И найдутся же дураки,кто это чмо купит.",0
3,"Из бисера делаю эти подснежники,а хотелось бы ...",0
4,"Да пристрелить уже этого педика ,из ума выжил ...",0


In [6]:
texts = []
labels = []

for idx in range(data.content.shape[0]):
    try:
        if len(data.label[idx]) == 1:
            text = BeautifulSoup(data.content[idx])
            # print(data.label[idx])
            texts.append(clean_str(text.get_text()))
            labels.append(data.label[idx])
    except:
        print(idx)
    

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

379
2626
2680
2681
2682
2683
2684
2685
2686
2687
2688
2689
2690
2691
2692
2693
2694
2705


/usr/local/lib/python3.8/dist-packages/keras/preprocessing/text.py:246: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn(


Found 13739 unique tokens.


In [8]:
data_pad = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print(('Shape of data tensor:', data_pad.shape))
print(('Shape of label tensor:', labels.shape))

indices = np.arange(data_pad.shape[0])
np.random.shuffle(indices)
data_pad = data_pad[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data_pad.shape[0])

x_train = data_pad[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data_pad[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set ')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
('Shape of data tensor:', (2694, 1000))
('Shape of label tensor:', (2694, 6))
Number of positive and negative reviews in traing and validation set 
[1931.   26.  176.   10.    3.   10.]
[484.   4.  43.   2.   1.   4.]


In [68]:
y_val

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [54]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
RMDL: Random Multimodel Deep Learning for Classification
 * Copyright (C) 2018  Kamran Kowsari <kk7nc@virginia.edu>
 * Last Update: 04/25/2018
 * This file is part of  RMDL project, University of Virginia.
 * Free to use, change, share and distribute source code of RMDL
 * Refrenced paper : RMDL: Random Multimodel Deep Learning for Classification
 * Refrenced paper : An Improvement of Data Classification using Random Multimodel Deep Learning (RMDL)
 * Comments and Error: email: kk7nc@virginia.edu
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''


from __future__ import print_function

import os, sys, tarfile
import numpy as np
import zipfile

if sys.version_info >= (3, 0, 0):
    import urllib.request as urllib  # ugly but works
else:
    import urllib

print(sys.version_info)

# image shape


# path to the directory with the data


# url of the binary data



# path to the binary train file with image data


def download_and_extract(data='Wikipedia'):
    """
    Download and extract the GloVe
    :return: None
    """

    if data=='Wikipedia':
        DATA_URL = 'http://nlp.stanford.edu/data/glove.6B.zip'
    elif data=='Common_Crawl_840B':
        DATA_URL = 'http://nlp.stanford.edu/data/wordvecs/glove.840B.300d.zip'
    elif data=='Common_Crawl_42B':
        DATA_URL = 'http://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip'
    elif data=='Twitter':
        DATA_URL = 'http://nlp.stanford.edu/data/wordvecs/glove.twitter.27B.zip'
    else:
        print("prameter should be Twitter, Common_Crawl_42B, Common_Crawl_840B, or Wikipedia")
        exit(0)


    dest_directory = DATA_DIR
    if not os.path.exists(dest_directory):
        os.makedirs(dest_directory)
    filename = DATA_URL.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)
    print(filepath)

    path = os.path.abspath(dest_directory)
    if not os.path.exists(filepath):
        def _progress(count, block_size, total_size):
            sys.stdout.write('\rDownloading %s %.2f%%' % (filename,
                                                          float(count * block_size) / float(total_size) * 100.0))
            sys.stdout.flush()

        filepath, _ = urllib.urlretrieve(DATA_URL, filepath)#, reporthook=_progress)


        zip_ref = zipfile.ZipFile(filepath, 'r')
        zip_ref.extractall(DATA_DIR)
        zip_ref.close()
    return path
download_and_extract()

sys.version_info(major=3, minor=8, micro=10, releaselevel='final', serial=0)
/content/drive/MyDrive/Colab Notebooks/диплом/glove.6B.zip


'/content/drive/MyDrive/Colab Notebooks/диплом'

In [70]:
print('shape x_train', x_train.shape)
print('shape y_train', y_train.shape)
print('shape x_val', x_val.shape)
print('shape y_val', y_val.shape)

shape x_train (2156, 1000)
shape y_train (2156, 6)
shape x_val (538, 1000)
shape y_val (538, 6)


In [14]:
# GLOVE_DIR = "/ext/home/analyst/Testground/data/glove"
embeddings_index = {}
f = open(os.path.join(DATA_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(6, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - simplified convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=128)

Total 400000 word vectors in Glove 6B 100d.
model fitting - simplified convolutional neural network
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1000)]            0         
                                                                 
 embedding_3 (Embedding)     (None, 1000, 100)         1374000   
                                                                 
 conv1d_9 (Conv1D)           (None, 996, 128)          64128     
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 199, 128)         0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 195, 128)          82048     
                                                                 
 max_pooling1d_10 (MaxPoo

In [20]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(filters=128, kernel_size=fsz, activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
    
l_merge = Add()(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(6, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=20, batch_size=50)

model fitting - more complex convolutional neural network
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 1000)]       0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 1000, 100)    1374000     ['input_7[0][0]']                
                                                                                                  
 conv1d_15 (Conv1D)             (None, 998, 128)     38528       ['embedding_6[0][0]']            
                                                                                                  
 conv1d_16 (Conv1D)             (None, 997, 128)     51328       ['embedding_6[0][0]']            
                                  